In [9]:
!pip install -q crewai ddgs ollama


In [10]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from ddgs import DDGS
import os


In [11]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama pull llama3


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
ERROR: This version requires zstd for extraction. Please install zstd and try again:
  - Debian/Ubuntu: sudo apt-get install zstd
  - RHEL/CentOS/Fedora: sudo dnf install zstd
  - Arch: sudo pacman -S zstd
/bin/bash: line 1: ollama: command not found


In [12]:
@tool("Medical literature search")
def medical_search(query: str) -> str:
    """Search recent medical and clinical trial information"""
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=5):
            results.append(f"- {r['title']}: {r['body']}")
    return "\n".join(results)


In [13]:
import os
os.environ["GROQ_API_KEY"] = "gsk_gf4eSrlRQNXbFdPfVv********************"

MODEL_NAME = "groq/llama-3.3-70b-versatile"

researcher = Agent(
    role="Senior Medical Researcher",
    goal="Find recent (2024–2026) ischemic stroke clinical trials",
    backstory="Expert in medical literature and clinical research",
    tools=[medical_search],
    verbose=True,
    llm=MODEL_NAME
)

analyst = Agent(
    role="Clinical Analyst",
    goal="Evaluate efficacy and safety of treatments",
    backstory="Specialist in clinical data interpretation",
    verbose=True,
    llm=MODEL_NAME
)

writer = Agent(
    role="Medical Communicator",
    goal="Write a professional medical report in Markdown",
    backstory="Experienced medical writer",
    verbose=True,
    llm=MODEL_NAME
)


In [14]:
research_task = Task(
    description="Research latest ischemic stroke treatments and clinical trials (2024–2026)",
    expected_output="Bullet-point research findings",
    agent=researcher
)

analysis_task = Task(
    description="Analyze efficacy, safety, and clinical relevance",
    expected_output="Clinical analysis summary",
    agent=analyst
)

writing_task = Task(
    description="Generate a structured Markdown medical report",
    expected_output="Markdown report",
    agent=writer
)


In [15]:
crew = Crew(
    agents=[researcher, analyst, writer],
    tasks=[research_task, analysis_task, writing_task],
    process=Process.sequential,
    verbose=True
)


In [16]:
result = crew.kickoff()
print(result)


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  a4b9d4a2-6c06-4acb-80dd-756aa1d2410d                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Research latest ischemic stroke treatments and clinical trials (2024–2026)                               │
│  ID: 7a104802-a53b-42c7-a777-c6f6d60f5762                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Medical Researcher                                                                               │
│                                                                                                                 │
│  Task: Research latest ischemic stroke treatments and clinical trials (2024–2026)                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#2) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: medical_literature_search                                                                                │
│  Args: {'query': 'ischemic stroke clinical trials 2024-2026'}                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

10:20:18 - LiteLLM:ERROR: litellm_logging.py:4483 - Error creating standard logging object - No module named 'fastapi_sso'
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/litellm/litellm_core_utils/litellm_logging.py", line 4370, in get_standard_logging_object_payload
    clean_metadata = StandardLoggingPayloadSetup.get_standard_logging_metadata(
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/litellm/litellm_core_utils/litellm_logging.py", line 3921, in get_standard_logging_metadata
    cold_storage_object_key = StandardLoggingPayloadSetup._generate_cold_storage_object_key(
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/litellm/litellm_core_utils/litellm_logging.py", line 4109, in _generate_cold_storage_object_key
    configured_cold_storage_logger = ColdStorageHandler._get_confi

...

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Generate a structured Markdown medical report                                                                  │
│  Agent:                                                                                                         │
│  Medical Communicator                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:LiteLLM:Error creating standard logging object - No module named 'fastapi_sso'
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/litellm/litellm_core_utils/litellm_logging.py", line 4370, in get_standard_logging_object_payload
    clean_metadata = StandardLoggingPayloadSetup.get_standard_logging_metadata(
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/litellm/litellm_core_utils/litellm_logging.py", line 3921, in get_standard_logging_metadata
    cold_storage_object_key = StandardLoggingPayloadSetup._generate_cold_storage_object_key(
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/litellm/litellm_core_utils/litellm_logging.py", line 4109, in _generate_cold_storage_object_key
    configured_cold_storage_logger = ColdStorageHandler._get_configured_cold_storage_custom_logger()
   

[CrewAIEventsBus] Warning: Event pairing mismatch. 'crew_kickoff_completed' closed 'task_started' (expected 
'crew_kickoff_started')

# Clinical Analysis Summary
The 2026 Guideline for the Early Management of Patients With Acute Ischemic Stroke, published by the American Heart Association/American Stroke Association, provides evidence-based recommendations for the management of acute ischemic stroke, from prehospital evaluation to acute treatment and early in-hospital management of complications. This guideline reflects the latest advances in stroke treatment, including expanded eligibility for clot-removal procedures, new evidence supporting the use of tenecteplase, and the implementation of mobile stroke units to deliver care faster and reduce the time to treatment.

## Efficacy
The guideline highlights the importance of early intervention in acute ischemic stroke, with a focus on rapid evaluation and treatment. The use of intravenous thrombolysis, endovascular thrombectomy, and blood pressure control after endovascular thrombectomy are discussed as key treatments for acute ischemic stroke. The Golden Bridge II tri

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  a4b9d4a2-6c06-4acb-80dd-756aa1d2410d                                                                           │
│  Final Output: # Clinical Analysis Summary                                                                      │
│  The 2026 Guideline for the Early Management of Patients With Acute Ischemic Stroke, published by the American  │
│  Heart Association/American Stroke Association, provides evidence-based recommendations for the management of   │
│  acute ischemic stroke, from prehospital evaluation to acute treatment and early in-hospital management of      │
│  complications. This guideline reflects the latest advances in stroke treatment, including expanded             │
│  eligibility for clot-removal procedures, new evidence supporting the use of tenecteplase, and the              │
│  implementation of mobile stroke units to deliver care faster and reduce the time to treatment.                 │
│                                                                                                                 │
│  ## Efficacy                                                                                                    │
│  The guideline highlights the importance of early intervention in acute ischemic stroke, with a focus on rapid  │
│  evaluation and treatment. The use of intravenous thrombolysis, endovascular thrombectomy, and blood pressure   │
│  control after endovascular thrombectomy are discussed as key treatments for acute ischemic stroke. The Golden  │
│  Bridge II trial, a cluster-randomized clinical trial, demonstrated the effectiveness of an artificial          │
│  intelligence-based clinical decision support system in improving stroke care quality and outcomes in patients  │
│  with acute ischemic stroke.                                                                                    │
│                                                                                                                 │
│  ## Safety                                                                                                      │
│  The guideline emphasizes the importance of careful patient selection and monitoring to minimize the risks      │
│  associated with stroke treatment. The use of tenecteplase, a clot-busting medication, is supported by new      │
│  evidence, but its safety profile must be carefully considered. The guideline also highlights the importance    │
│  of blood pressure control after endovascular thrombectomy to minimize the risk of complications.               │
│                                                                                                                 │
│  ## Clinical Relevance                                                                                          │
│  The 2026 Guideline for the Early Management of Patients With Acute Ischemic Stroke has significant clinical    │
│  relevance, as it provides healthcare professionals with the latest evidence-based recommendations for the      │
│  management of acute ischemic stroke. The expanded eligibility for clot-removal procedures and the use of       │
│  tenecteplase have the potential to improve outcomes for patients with acute ischemic stroke. The               │
│  implementation of mobile stroke units is also expecte

In [19]:
with open("report.md", "w") as f:
    f.write(str(result))

print("✅ report.md generated successfully")


✅ report.md generated successfully


In [20]:
from google.colab import files
files.download("report.md")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>